In [1]:
import os
import sys
import math
import argparse
import numpy as np
from collections import Counter

# Keras imports
from keras import layers
from keras import backend as K
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Permute, Reshape, multiply, concatenate, RepeatVector

# ML4CVD Imports
from ml4cvd.arguments import parse_args
from ml4cvd.models import make_multimodal_to_multilabel_model, train_model_from_generators

# IPython imports
from IPython.display import Image
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


In [2]:
with open('/home/sam/europarl-v7.es-en.en', 'r', encoding='utf-8') as english_file:
    with open('/home/sam/europarl-v7.es-en.es', 'r', encoding='utf-8') as spanish_file:
        english = english_file.read()
        spanish = spanish_file.read()

In [3]:
start_char = '\t'
end_char = '\n'
english_lines = english.split('\n')
spanish_lines = [start_char + l + end_char for l in spanish.split('\n')]

In [4]:
english_chars = sorted(list(set(english)))
print('total english chars:', len(english_chars))
spanish_chars = sorted(list(set(spanish + start_char + end_char)))
print('total spanish chars:', len(spanish_chars))
chars = sorted(list(set(english_chars + spanish_chars)))
print('total chars:', len(chars))

total english chars: 314
total spanish chars: 334
total chars: 339


In [5]:
max_english = 100 # max(map(len, english_lines))//2
max_spanish = 100 # max(map(len, spanish_lines))//2
max_pairs = 15000

In [6]:
english_token_index = {char: i for i, char in enumerate(english_chars)}
spanish_token_index = {char: i for i, char in enumerate(spanish_chars)}
english_index_tokens = {i: char for i, char in enumerate(english_chars)}
spanish_index_tokens = {i: char for i, char in enumerate(spanish_chars)}
encoder_input_data = np.zeros((max_pairs, max_english, len(english_chars)), dtype='float32')
decoder_input_data = np.zeros((max_pairs, max_spanish, len(spanish_chars)), dtype='float32')
decoder_target_data = np.zeros((max_pairs, max_spanish, len(spanish_chars)), dtype='float32')

for i, (english_text, spanish_text) in enumerate(zip(english_lines, spanish_lines)):
    for t, char in enumerate(english_text):
        if t == max_english:
            break
        encoder_input_data[i, t, english_token_index[char]] = 1.
    for t, char in enumerate(spanish_text):
        if t == max_spanish:
            break
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, spanish_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep and will not include the start character.
            decoder_target_data[i, t - 1, spanish_token_index[char]] = 1.
    if i == max_pairs-1:
        break

In [37]:
latent_dim = 256
batch_size = 64
epochs = 6

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, len(english_chars)))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, len(spanish_chars)))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(len(spanish_chars), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 12000 samples, validate on 3000 samples
Epoch 1/6
12000/12000 [==============================] - 65s 5ms/step - loss: 2.5933 - val_loss: 2.2855
Epoch 2/6
12000/12000 [==============================] - 63s 5ms/step - loss: 2.1065 - val_loss: 1.9738
Epoch 3/6
12000/12000 [==============================] - 62s 5ms/step - loss: 1.8755 - val_loss: 1.8111
Epoch 4/6
12000/12000 [==============================] - 60s 5ms/step - loss: 1.7421 - val_loss: 1.7119
Epoch 5/6
12000/12000 [==============================] - 60s 5ms/step - loss: 1.6335 - val_loss: 1.5995
Epoch 6/6
12000/12000 [==============================] - 61s 5ms/step - loss: 1.5425 - val_loss: 1.5143


/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_8 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_7/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_7/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [38]:

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [68]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ~~~~~ Sample with Temperature ~~~~~
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def sample(preds, temperature=1.0):
	# helper function to sample an index from a probability array
	preds = np.asarray(preds).astype('float64')
	preds = np.log(preds) / temperature
	exp_preds = np.exp(preds)
	preds = exp_preds / np.sum(exp_preds)
	probas = np.random.multinomial(1, preds, 1)
	return np.argmax(probas)


In [39]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, len(spanish_chars)))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, spanish_token_index[start_char]] = 1.

    # Sampling loop for a batch of sequences (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = spanish_index_tokens[sampled_token_index]
        decoded_sentence += sampled_char
        #print('sampled char is:', sampled_token_index, sampled_char)
        # Exit condition: either hit max length or find stop character.
        if (sampled_char == end_char or len(decoded_sentence) > max_spanish):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, len(spanish_chars)))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [40]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', english_lines[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('Actual sentence:', spanish_lines[seq_index])

-
Input sentence: Resumption of the session
Decoded sentence: Es el prosento de la Comisión de la Comisión de la Comisión de la Comisión de la Comisión de la Comis
Actual sentence: 	Reanudación del período de sesiones

-
Input sentence: I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
Decoded sentence: En esta esta esta esta esta esta esta en el prober los de la Comisión de la Comisión de la Comisión d
Actual sentence: 	Declaro reanudado el período de sesiones del Parlamento Europeo, interrumpido el viernes 17 de diciembre pasado, y reitero a Sus Señorías mi deseo de que hayan tenido unas buenas vacaciones.

-
Input sentence: Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.
Decoded sentenc

In [71]:
def attention_3d_block(inputs, time_steps):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, time_steps))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(time_steps, activation='softmax')(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = multiply([inputs, a_probs], name='attention_mul')
    return output_attention_mul

In [72]:
latent_dim = 256
batch_size = 64
epochs = 96
from keras.layers import RepeatVector, concatenate
# Define an input sequence and process it.
encoder_inputs = Input(shape=(max_english, len(english_chars)))
encoder_out = LSTM(latent_dim)(encoder_inputs)
encoder_repeat = RepeatVector(max_spanish)(encoder_out)
attended_encoding = attention_3d_block(encoder_repeat, max_spanish)

burn_in = Input(shape=(max_spanish, len(spanish_chars)))
lstm_in = layers.concatenate([burn_in, attended_encoding], name='concat_spanish_and_attended')
decoder_lstm = LSTM(latent_dim, return_sequences=True)
decoder_outputs = decoder_lstm(lstm_in)
decoder_outputs = Dense(len(spanish_chars), activation='softmax')(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, burn_in], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           (None, 100, 314)     0                                            
__________________________________________________________________________________________________
lstm_21 (LSTM)                  (None, 256)          584704      input_33[0][0]                   
__________________________________________________________________________________________________
repeat_vector_11 (RepeatVector) (None, 100, 256)     0           lstm_21[0][0]                    
__________________________________________________________________________________________________
permute_11 (Permute)            (None, 256, 100)     0           repeat_vector_11[0][0]           
__________________________________________________________________________________________________
reshape_11

In [73]:
model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)
# Save model
model.save('s2s_attention_96.h5')

Train on 12000 samples, validate on 3000 samples
Epoch 1/96
12000/12000 [==============================] - 83s 7ms/step - loss: 2.5907 - val_loss: 2.3498
Epoch 2/96
12000/12000 [==============================] - 77s 6ms/step - loss: 2.1431 - val_loss: 1.9717
Epoch 3/96
12000/12000 [==============================] - 78s 6ms/step - loss: 1.8675 - val_loss: 1.7837
Epoch 4/96
12000/12000 [==============================] - 79s 7ms/step - loss: 1.7024 - val_loss: 1.6468
Epoch 5/96
12000/12000 [==============================] - 78s 6ms/step - loss: 1.5723 - val_loss: 1.5244
Epoch 6/96
12000/12000 [==============================] - 76s 6ms/step - loss: 1.4676 - val_loss: 1.4438
Epoch 7/96
12000/12000 [==============================] - 76s 6ms/step - loss: 1.3805 - val_loss: 1.3670
Epoch 8/96
12000/12000 [==============================] - 77s 6ms/step - loss: 1.3089 - val_loss: 1.3054
Epoch 9/96
12000/12000 [==============================] - 80s 7ms/step - loss: 1.2486 - val_loss: 1.2621
Epoch 

Epoch 78/96
12000/12000 [==============================] - 70s 6ms/step - loss: 0.6914 - val_loss: 1.0543
Epoch 79/96
12000/12000 [==============================] - 69s 6ms/step - loss: 0.6895 - val_loss: 1.0536
Epoch 80/96
12000/12000 [==============================] - 70s 6ms/step - loss: 0.6875 - val_loss: 1.0564
Epoch 81/96
12000/12000 [==============================] - 69s 6ms/step - loss: 0.6856 - val_loss: 1.0628
Epoch 82/96
12000/12000 [==============================] - 69s 6ms/step - loss: 0.6837 - val_loss: 1.0610
Epoch 83/96
12000/12000 [==============================] - 69s 6ms/step - loss: 0.6818 - val_loss: 1.0664
Epoch 84/96
12000/12000 [==============================] - 69s 6ms/step - loss: 0.6801 - val_loss: 1.0660
Epoch 85/96
12000/12000 [==============================] - 70s 6ms/step - loss: 0.6780 - val_loss: 1.0652
Epoch 86/96
12000/12000 [==============================] - 69s 6ms/step - loss: 0.6765 - val_loss: 1.0740
Epoch 87/96
12000/12000 [=====================

In [74]:
# Define sampling models
attn_encoder_model = Model(encoder_inputs, attended_encoding)
attn_encoder_model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           (None, 100, 314)     0                                            
__________________________________________________________________________________________________
lstm_21 (LSTM)                  (None, 256)          584704      input_33[0][0]                   
__________________________________________________________________________________________________
repeat_vector_11 (RepeatVector) (None, 100, 256)     0           lstm_21[0][0]                    
__________________________________________________________________________________________________
permute_11 (Permute)            (None, 256, 100)     0           repeat_vector_11[0][0]           
__________________________________________________________________________________________________
reshape_11

In [75]:
def decode_attended_sequence(input_seq):
    # Encode the input as state vectors.
    attended_np = attn_encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    burn_in_np = np.zeros((1, max_spanish, len(spanish_chars)))
    # Populate the first character of target sequence with the start character.
    burn_in_np[0, 0, spanish_token_index[start_char]] = 1.

    # Sampling loop for a batch of sequences (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    char_idx = 0
    while not stop_condition:
        char_idx += 1
        output_tokens = model.predict([input_seq, burn_in_np])
        # Sample a token
        sampled_token_index =  sample(output_tokens[0, char_idx-1, :], 1.1)
        sampled_char = spanish_index_tokens[sampled_token_index]
        decoded_sentence += sampled_char

        # Update the target sequence (of length 1).
        burn_in_np[0, char_idx, sampled_token_index] = 1.
        
        
        # Exit condition: either hit max length or find stop character.
        if (sampled_char == end_char or len(decoded_sentence) > max_spanish) or char_idx == max_spanish-1:
            stop_condition = True

    return decoded_sentence

In [76]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_attended_sequence(input_seq)
    print('-')
    print('Input sentence:', english_lines[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('Actual sentence:', spanish_lines[seq_index])

-
Input sentence: Resumption of the session
Decoded sentence: La competencia tiene la aniza entre los productores abrerán muy blegada los Estados miembros.

Actual sentence: 	Reanudación del período de sesiones

-
Input sentence: I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
Decoded sentence: Esto recorda muy sustancialidad del Convenio debe considerarse vivilantes de la formulidad de agua.
Actual sentence: 	Declaro reanudado el período de sesiones del Parlamento Europeo, interrumpido el viernes 17 de diciembre pasado, y reitero a Sus Señorías mi deseo de que hayan tenido unas buenas vacaciones.

-
Input sentence: Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.
Decoded sentence: Entreg